# Checking Out Dataset

In [1]:
import pandas as pd

In [4]:
# Loading data
df = pd.read_csv('./data/us_car_data.csv')
df

,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,CAFV,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Electric Utility,2020 Census Tract,Longitude,latitude
0,Thurston,Tumwater,WA,98512,2019,TESLA,MODEL 3,Battery Electric Vehicle,Clean Alternative Fuel Vehicle Eligible,220,0,22,242565116,PUGET SOUND ENERGY INC,53067010910,-122.913102,47.013593
1,Island,Clinton,WA,98236,2022,NISSAN,LEAF,Battery Electric Vehicle,Eligibility unknown as battery range has not b...,0,0,10,183272785,PUGET SOUND ENERGY INC,53029972000,-122.359364,47.979655
2,Snohomish,Snohomish,WA,98290,2020,TESLA,MODEL 3,Battery Electric Vehicle,Clean Alternative Fuel Vehicle Eligible,266,0,44,112552366,PUGET SOUND ENERGY INC,53061052502,-122.091505,47.915555
3,King,Seattle,WA,98134,2020,TESLA,MODEL 3,Battery Electric Vehicle,Clean Alternative Fuel Vehicle Eligible,322,0,11,6336319,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033009300,-122.329815,47.579810
4,Snohomish,Edmonds,WA,98020,2013,TESLA,MODEL S,Battery Electric Vehicle,Clean Alternative Fuel Vehicle Eligible,208,69900,21,186212960,PUGET SOUND ENERGY INC,53061050403,-122.375070,47.808070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134469,King,Yarrow Point,WA,98004,2020,TESLA,MODEL S,Battery Electric Vehicle,Clean Alternative Fuel Vehicle Eligible,330,0,48,124776627,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033024100,-122.201905,47.613850
134470,King,Burien,WA,98168,2021,TESLA,MODEL Y,Battery Electric Vehicle,Eligibility unknown as battery range has not b...,0,0,33,142857676,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033027100,-122.286460,47.476130
134471,Pierce,Gig Harbor,WA,98335,2022,TESLA,MODEL S,Battery Electric Vehicle,Eligibility unknown as battery range has not b...,0,0,26,220157724,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,53053072407,-122.583545,47.323449
134472,Grant,Ephrata,WA,98823,2023,HYUNDAI,IONIQ 5,Battery Electric Vehicle,Eligibility unknown as battery range has not b...,0,0,13,223872596,PUD NO 2 OF GRANT COUNTY,53025010402,-119.555130,47.319360
